In [1]:
import numpy as np
import xarray as xr
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
from src.score import *
import re

import generate_data as gd

In [9]:
def build_dnn(input_shape):
    """Fully convolutional network"""
    x = input = Input(shape=input_shape)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    x = Dense(256)(x)
    x = LeakyReLU()(x)
    output = Dense(2)(x)
    return keras.models.Model(input, output)

def create_predictions(model, dg):
    """Create non-iterative predictions"""
    preds_un = xr.DataArray(
        model.predict(dg),
        dims=['time', 'lat', 'lon', 'level'],
        coords={'time': dg.valid_time, 'lat': dg.data.lat, 'lon': dg.data.lon,
                'level': dg.data.isel(level=dg.output_idxs).level,
                'level_names': dg.data.isel(level=dg.output_idxs).level_names
               },
    )
    # Unnormalize
    preds = (preds_un * dg.std.isel(level=dg.output_idxs).values +
             dg.mean.isel(level=dg.output_idxs).values)
    unique_vars = list(set([l.split('_')[0] for l in preds.level_names.values])); unique_vars

    das = []
    for v in unique_vars:
        idxs = [i for i, vv in enumerate(preds.level_names.values) if vv.split('_')[0] in v]
        #print(v, idxs)
        da = preds.isel(level=idxs).squeeze().drop('level_names')
        if not 'level' in da.dims: da.drop('level')
        das.append({v: da})
    return preds_un, xr.merge(das, compat = 'override').drop('level')

In [10]:
def predictions(var_name, valid):
    dg_train, dg_valid, dg_test = gd.create_data(var_name)

    if var_name == 'specific_humidity':
        cnn = build_dnn((32, 64, 4))
    elif var_name == 'pot_vort':
        cnn = build_dnn((32, 64, 4))
    elif var_name == 'const':
        cnn = build_dnn((32, 64, 5))
    elif var_name == 'orig':
        cnn = build_dnn((32, 64, 2))
    else:
        cnn = build_dnn((32, 64, 3))

    cnn.compile(keras.optimizers.Adam(1e-4), 'mse')

    cnn.load_weights('/rds/general/user/mc4117/home/WeatherBench/saved_models/72_gpu_annual_dense_' + str(var_name) + '.h5')

    number_of_forecasts = 1

    pred_ensemble=np.ndarray(shape=(2, 17448, 32, 64, number_of_forecasts),dtype=np.float32)
    print(pred_ensemble.shape)
    forecast_counter=np.zeros(number_of_forecasts,dtype=int)

    for j in range(number_of_forecasts):
        print(j)
        preds, output = create_predictions(cnn, dg_test)
        pred2 = np.asarray(output.to_array(), dtype=np.float32).squeeze()
        pred_ensemble[:,:,:,:,j]=pred2
        forecast_counter[j]=j+1
        filename2 = '/rds/general/user/mc4117/ephemeral/saved_pred/72_gpu_annual' + str(var_name)
        np.save(filename2 + '.npy', pred_ensemble)
        


    return preds, compute_weighted_rmse(output, valid).compute()

In [11]:
DATADIR = '/rds/general/user/mc4117/home/WeatherBench/data/'

z500_valid = load_test_data(f'{DATADIR}geopotential_500', 'z')
t850_valid = load_test_data(f'{DATADIR}temperature_850', 't')
valid = xr.merge([z500_valid, t850_valid])


In [12]:
preds_orig, rmse = predictions('orig', valid)
rmse

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 910.6
    t        float64 4.025

In [17]:
preds_sh, rmse_sh = predictions('specific_humidity', valid)
rmse_sh

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 887.7
    t        float64 3.96

In [13]:
preds_pv, rmse_pv = predictions('pot_vort', valid)
rmse_pv

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 894.3
    t        float64 3.952

In [14]:
preds_const, rmse_const = predictions('const', valid)
rmse_const

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 826.1
    t        float64 3.796

In [15]:
preds_2mt, rmse_2mt = predictions('2m temp', valid)
rmse_2mt

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 862.0
    t        float64 3.797

In [16]:
preds_sr, rmse_sr = predictions('solar rad', valid)
rmse_sr

Loading data into RAM
Loading data into RAM
Loading data into RAM
(2, 17448, 32, 64, 1)
0


<xarray.Dataset>
Dimensions:  ()
Data variables:
    z        float64 908.6
    t        float64 4.008